In [ ]:
from platform import python_version
print(python_version())

## Super-summary: chosen cutoffs

### Definitions:
  - LFC table has:
    - abs_LFC (the absolute LFC cutoff value)
    - FDR_LFC, its FDR or p-value adjusted
  - The enriched pathway table has:
    - FDR_pathway cutoff value
   
### Default values for LFC table:
  - abs_LFC = 1
  - FDR_LFC = 0.05
  - therefore, a DEG/DAP is abs(LFC) >= 1 and FDR < 0.05

### Default values for Enriched Pathways:
  - FDR_pathway = 0.05
  - therefore, an enriched pathways has FDR < 0.05 and at least 3 DEGs/DAPs

### Calculating the best cutoffs:
  - We proposed and calculated many toies to define a new statistics to flebilize the LFC and Enriched Pathway cutoffs.
    - toies are calculated for each case, each cutoff, and each resulting enriched pathway.
  - To find the possible best LFC/FDR expression and FDR pathway cutoffs:
     - We look for a high number in n_pathway and n_DEGs_in_pathway, having a low FDR_LFC and a high absLFC.
       - The default FDR_LFC (0.05):
          - It may have fewer DEGs, resulting in fewer enriched pathways.
          - It may have fewer enriched pathways, even having many DEGs/DAPs.
       - Therefore, a trade-off exists between optimizing (abs_LFC and FDR_LFC cutoffs) and (FDR_pathway cutoffs, n_pathways, and n_DEGs_in_pathways.)

### An toi measures the trade-off between "LFC" and "Enriched Pathways" cufoff -> LFC - Enriched Pathway Trade-Off Statistics (LEATOS)

  - We proposed and calculated the following possible toies:

<p style="font-size: 20px; color: lightgreen;">
$toi1 = \sqrt{-log{_{10}}{FDR_{pathway}} * \frac{n}{N} }$ </p>

<p style="font-size: 20px; color: cyan;">
$toi2 = \sqrt{-log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} }$ </p>

<p style="font-size: 20px; color: orange;">
$toi3 = (-log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} * \frac{n}{N})^{1/3}$ </p>

<p style="font-size: 20px; color: pink;">
$toi4 = (abs\_LFC * -log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} * \frac{n}{N})^{1/4}$ </p>

where,
  - n is the number of DEGs/DAPs found in the pathway
  - N is the total number of annotated DEGs/DAPs in the pathway (DAPend in the database, our default database is Reactome 2022)

### Then we searched for the best cutoffs
  - In each 5 percentile of the toi histogram, we look for the best abs_LFC, FDR_LFC, FDR_pathway:
  -  We expected that the best cutoff should be in the right tail of the histogram (high toi value.)
  -  High toi values must have a high number of n_pathways and n DEGs in pathways.

### Testing the best cutoffs (for each case)

  - Is the new set of cutoffs correct? good enough?
  - How to establish that the calculated cutoff is correct?
  - To answer these questions we calculated the chi-square test between the "best cutoff" and the "default"
    - Best cutoff has:
      - n DEGs/DAPs in pathways
      - n DEGs/DAPs not in pathways
    - The Default cutoff may have:
      - n DEGs/DAPs in pathways
      - n DEGs/DAPs not in pathways
      - The DEGs/DAPs can be:
        - greater or equal number of the best cutoff DEGs/DAPs
        - fewer number of the best cutoff DEGs/DAPs:
           - in this case, one complements the number of DEGs/DAPs with random genes not DEGs/DAPs (found in the experiment)

#### Chi-square test:

DEGs/DAPs | # in pathway | no in pathway
--- | --- | --- 
 Best cutoff |     A      |   B  
 Default cutoff |   C | D 

Chi-square p-value:
  - p-value < 0.05 denotes that both distributions are not similar; therefore, random genes could not reach the best cutoff DEGs/DAPs; in conclusion, the best cutoff was not found randomly.
  - p-value \>= 0.05 denotes that both distributions are similar, and the best cutoff can be achieved randomly.

In [ ]:
import os, sys, pickle

import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', 80)
pd.set_option("display.precision", 3)

import yaml

import seaborn as sns
sns.set_context("notebook", font_scale=1.4)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

sys.path.insert(1, '../src/')

from Basic import *
from enricher_lib import *
from config_lib import *

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

# !pip3 install pyyaml
with open('params.yml', 'r') as file:
    dic_yml = yaml.safe_load(file)

In [ ]:
root_chibe = dic_yml['root_chibe']
root_colab = dic_yml['root_colab']
root0 = dic_yml['root0']

project = dic_yml['project']
s_project = dic_yml['s_project']

email = dic_yml['email']

gene_protein = dic_yml['gene_protein']
s_omics = dic_yml['s_omics']

has_age = dic_yml['has_age']
has_gender = dic_yml['has_gender']

want_normalized = dic_yml['want_normalized']

abs_lfc_cutoff_inf = dic_yml['abs_lfc_cutoff_inf']
s_pathw_enrichm_method = dic_yml['s_pathw_enrichm_method']
num_min_degs_for_ptw_enr = dic_yml['num_min_degs_for_ptw_enr']

tolerance_pathway_index = dic_yml['tolerance_pathway_index']
type_sat_ptw_index = dic_yml['type_sat_ptw_index']
saturation_lfc_index = dic_yml['saturation_lfc_index']
chosen_model_sampling = dic_yml['chosen_model_sampling']

case_list = dic_yml['case_list']

pval_pathway_cutoff = dic_yml['pval_pathway_cutoff']
fdr_pathway_cutoff = dic_yml['fdr_pathway_cutoff']
num_of_genes_cutoff = dic_yml['num_of_genes_cutoff']

run_list = dic_yml['run_list']
chosen_model_list = dic_yml['chosen_model_list']
i_dfp_list = dic_yml['i_dfp_list']

exp_normalization='quantile_norm' if want_normalized else None
normalization='not_normalized' if exp_normalization is None else exp_normalization

cfg = Config(project, s_project, case_list, root0)

case = case_list[0]

n_genes_annot_ptw, n_degs, n_degs_in_ptw, n_degs_not_in_ptw, degs_in_all_ratio = -1,-1,-1,-1,-1
abs_lfc_cutoff, fdr_lfc_cutoff, n_degs, n_degs_up, n_degs_dw = cfg.get_best_lfc_cutoff(case, 'not_normalized')


print(f"G/P LFC cutoffs: lfc={abs_lfc_cutoff:.3f}; fdr={fdr_lfc_cutoff:.3f}")
print(f"Pathway cutoffs: pval={pval_pathway_cutoff:.3f}; fdr={fdr_pathway_cutoff:.3f}; num of genes={num_of_genes_cutoff}")

In [ ]:
enr = enricheR(gene_protein, s_omics, project, s_project, root0,
             case_list, has_age, has_gender, clone_objects=False,
             exp_normalization=exp_normalization, geneset_num=0, 
             num_min_degs_for_ptw_enr=num_min_degs_for_ptw_enr, 
             tolerance_pathway_index=tolerance_pathway_index, 
             s_pathw_enrichm_method = s_pathw_enrichm_method,
             abs_lfc_cutoff_inf = abs_lfc_cutoff_inf, 
             type_sat_ptw_index=type_sat_ptw_index, saturation_lfc_index=saturation_lfc_index)

case = case_list[0]

enr.cfg.set_default_best_lfc_cutoff(normalization, abs_lfc_cutoff=1, fdr_lfc_cutoff=0.05)
ret, degs, degs_ensembl, dfdegs = enr.open_case(case, verbose=False)
print("\nEcho Parameters:")
enr.echo_parameters()

geneset_num = enr.geneset_num

In [ ]:
case = case_list[1]
ret, degs, degs_ensembl, dfdegs = enr.open_case(case, verbose=False)
print("\nEcho Parameters:")
enr.echo_parameters()

### Distribution related to possible TOIs

In [ ]:
toi_cols = ['toi1_median',  'toi2_median','toi3_median',  'toi4_median',
              'n_pathways', 'n_degs_in_pathways', 'n_degs_in_pathways_mean', 'n_degs_in_pathways_median']

for col in toi_cols:
    fig = enr.plot_cutoff_simulation_histograms(col, width=1100, height=270)
    print(col)
    fig.show()

In [ ]:
dfsim = enr.open_simulation_table()
dfsim = dfsim.sort_values(['case', 'fdr_lfc_cutoff', 'abs_lfc_cutoff'], ascending=[True, False, False])

dfsim.head(2)

### First, per case plot
#### DAPs 'in pathway' x toi 4

In [ ]:
colors=['navy', 'red', 'darkcyan', 'darkgreen', 'orange', 'brown', 'darksalmon',
        'magenta', 'darkturquoise', 'orange', 'darkred', 'indigo', 'magenta', 'maroon', 'black',
        'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'olivedrab', 'navy'] + plotly_colors_proteins

cols = ['case', 'toi4_median', 'med_max_ptw',  'quantile', 'quantile_val_inf', 'quantile_val_sup',
        'abs_lfc_cutoff', 'fdr_lfc_cutoff', 'pathway_fdr_cutoff',  'n_pathways',
        'n_degs_in_pathways']  #  'toi1_median', 'toi2_median',  'toi3_median',

In [ ]:
selected_toi_col = 'toi4_median'

In [ ]:
fig = enr.plot_degs_in_pathways_vs_toi_per_case(selected_toi_col=selected_toi_col, title=None, plot_all_dfi=False,
                                                width=1100, height=600, sel_colors=None, plot_bgcolor='lightgray', verbose=False)

if fig: fig.show()

### Next, per case plot
#### ALL DAPs x abs_LFC
#### Comments:

  - each FDR selects a subset of DEG/DAPs
    - abs_LFC may or may not filter them
    - we define that only 'curves' that change DEG/DAPs varying the abs_LFC are valid
    - we observed that most of the curves saturate at 0.4 abs_LFC
  - we defined the range of FDR from 0.05 to 0.75 (here 0.55)
    - if we had defined:
      - min abs_LFC = 0 and max_FDR = 1, we got in the left-upper corner the total Proteins (or Genes) 

In [ ]:
cols = ['case', 'parameter', 'quantile', 'med_max_ptw', 'toi4_median', 'abs_lfc_cutoff', 'fdr_lfc_cutoff',
        'pathway_fdr_cutoff', 'n_pathways', 'n_degs_in_pathways', 
        'toi1_median',  'toi2_median', 'toi3_median']

### TOI4_median

In [ ]:
# selected_toi_col  ['toi1_median', 'toi2_median', 'toi3_median', 'toi4_median' ]

selected_toi_col = 'toi4_median'

fig_list = enr.plot_genes_and_pathways_frequecies_per_cases(selected_toi_col,  width=1100, height=700)

fig0 = fig_list[0]
fig1 = fig_list[1]
print(">>>", selected_toi_col)
print(f"# {enr.s_deg_dap}s")
fig0.show()
print("# n pathways")
fig1.show()

In [ ]:
# dfbest.columns

### Selected best cutoffs per case

In [ ]:
dfbest = enr.cfg.open_best_ptw_cutoff(verbose=False)
dfbest = dfbest[dfbest.med_max_ptw == 'median']
print(len(dfbest))

dfbest[cols]

## Why toi4_median is the best approach?

#### balance between best LFC cutoffs and Pathway cutoffs

### First using 'n_pathways' as toi
### Next using 'toi4_median' as toi
#### Any toi can be used to compare to toi4_median

In [ ]:
n_best_samples_chosen = 6

cols2 = ['case', 'med_max_ptw', 'quantile', 'toi4_median', 'toi4_mean', 'abs_lfc_cutoff', 'fdr_lfc_cutoff', 'pathway_fdr_cutoff', 
        'n_pathways', 'n_degs_in_pathways', 'n_degs_in_pathways_mean',
        'n_degs_in_pathways_median', 'n_degs_in_pathways_std',
        'toi1_median', 'toi2_median', 'toi3_median']

dfconfig = enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=n_best_samples_chosen, save_config=False, verbose=True)
dfconfig = dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

In [ ]:
dfconfig = enr.calc_best_cutoffs_params(selected_toi_col='toi4_median', n_best_sample=n_best_samples_chosen+1, save_config=False, verbose=True)
dfconfig = dfconfig[dfconfig.med_max_ptw == 'median']
dfconfig[cols2]

## It minimizes fdr and maximizes abs_lfc!!!

### Summary DAPs + Up and Down

In [ ]:
per_biotype=False
ensembl=False;
dfa = enr.summary_degs_up_down(per_biotype=per_biotype, ensembl=ensembl, verbose=False)
print(len(dfa))
dfa

In [ ]:
per_biotype=True
ensembl=False;
dfa = enr.summary_degs_up_down(per_biotype=per_biotype, ensembl=ensembl, verbose=False)
print(len(dfa))
dfa

In [ ]:
per_biotype=True
ensembl=True;
dfa = enr.summary_degs_up_down(per_biotype=per_biotype, ensembl=ensembl, verbose=False)
print(len(dfa))
dfa

In [ ]:
before_best_cutoff = False

per_biotype=False
ensembl=False

fig, dfa = enr.barplot_up_down_genes_per_case(per_biotype=per_biotype, ensembl=ensembl, before_best_cutoff=before_best_cutoff,
                                              width=1100, height=700, verbose=False)
fig.show()

In [ ]:
before_best_cutoff = False

per_biotype=True
ensembl=False

fig, dfa = enr.barplot_up_down_genes_per_case(per_biotype=per_biotype, ensembl=ensembl, before_best_cutoff=before_best_cutoff,
                                              width=1100, height=700, verbose=False)
fig.show()

In [ ]:
before_best_cutoff = False

per_biotype=True
ensembl=True

fig, dfa = enr.barplot_up_down_genes_per_case(per_biotype=per_biotype, ensembl=ensembl, before_best_cutoff=before_best_cutoff,
                                              width=1100, height=700, verbose=False)
fig.show()

### Sampling

In [ ]:
fig, dfa = enr.barplot_sampling_cutoffs(prompt_verbose=False, verbose=False)
fig.show()